In [11]:
# This code reads in a lensing galaxy catalogue and appends to it 
# values of the mass over-density and tidal field tensor at the galaxy locations
# By Joachim Harnois-Deraps, 2021.

import numpy as np
import healpy as hp
#from scipy.interpolate import interp1d
import random
import input
from pathlib import Path


# The tomographic bin
tomobin = 4

print("**************************")
print("Processing tomo bin",tomobin)
print("**************************")



# the number of galaxies per square arcminute to extract
gpam = 0.06

# The NSIDE value of the shear and weight maps
nside = 8192

npix = 12*nside**2
pix_size = np.sqrt(4.*np.pi*(180./np.pi*60)**2/npix)
print('pixel size of shear maps is %f arcmin per side' % pix_size)

zfile=np.loadtxt("z2ts.txt",delimiter=',')
z_list=np.flip(zfile[1:58,0])
snaplist = np.flip(zfile[1:58,1].astype(int))
n_slices = np.size(z_list)


**************************
Processing tomo bin 4
**************************
pixel size of shear maps is 0.429436 arcmin per side


In [12]:
#for plane in range(0,41):
for plane in range(0,n_slices):

    print('starting analysis of plane %i' % plane)
            
    #fname = input.GalDir+'StageIV_nz/GalCat_tomo'+np.str(tomobin)+'_plane'+np.str(plane)+'_'+np.str(gpam)+'GpAM_RA_Dec_g1_g2_w.asc'
    fname = input.GalDir+'StageIV_nz/GalCat_tomo'+np.str(tomobin)+'_plane'+np.str(plane)+'_'+np.str(gpam)+'GpAM_RA_Dec_g1_g2_w_linear_bias.asc'

    # check file exists:
    my_file = Path(fname)
    if my_file.is_file():
        # file exists
        print("Got file! Reading",fname)
        ra, dec, shear1, shear2, w, z = np.loadtxt(fname, unpack=True)
        
        #load mass maps and tidal tensor
        fname = input.deltaDir+"density_map_"+str(snaplist[plane])+"_dens_allsky.npy"
        #fname = input.deltaDir+"density_map_"+str(snaplist[plane])+"_dens_allsky_smooth.npy"

        print("opening ", fname) 
        hpmap_density = np.load(fname)
        print("Got density file!")
        
        #fname = input.tidalDir+"tidal_field_map_"+str(snaplist[plane])+"_allsky.npy"
        fname = input.tidalDir+"tidal_field_map_"+str(snaplist[plane])+"_allsky_V2.npy"

        print("opening ", fname) 
        hpmap_tidal = np.load(fname)
        print("Got tidal file!")
        

        # Normalize the maps correctly:       
        mean_rho = (np.mean(hpmap_density)) # no factor of 8.0 since we have full sky data here
        hpmap_density /= mean_rho 
        hpmap_density -= 1
        
        Norm_empirical = 0.6525 
        
        hpmap_tidal *= Norm_empirical

        print("over density:",np.min(hpmap_density), np.max(hpmap_density), np.mean(hpmap_density))
        print("s11:",np.min(hpmap_tidal[:,0]), np.max(hpmap_tidal[:,0]), np.mean(hpmap_tidal[:,0]))
        print("s22:",np.min(hpmap_tidal[:,1]), np.max(hpmap_tidal[:,1]), np.mean(hpmap_tidal[:,1]))
        print("s12:",np.min(hpmap_tidal[:,2]), np.max(hpmap_tidal[:,2]), np.mean(hpmap_tidal[:,2]))


        
        # Interpolate maps at galaxy positions
        den_pix = hp.get_interp_val(hpmap_density, (90.-dec)*np.pi/180., ra*np.pi/180., nest=False )

        s11_pix = hp.get_interp_val(hpmap_tidal[:,0], (90.-dec)*np.pi/180., ra*np.pi/180., nest=False )
        s22_pix = hp.get_interp_val(hpmap_tidal[:,1], (90.-dec)*np.pi/180., ra*np.pi/180., nest=False )
        s12_pix = hp.get_interp_val(hpmap_tidal[:,2], (90.-dec)*np.pi/180., ra*np.pi/180., nest=False )



        # Save a shear catalogue in standard format to be read by Athena
        #outname = input.GalDir+'StageIV_nz/GalCat_tomo'+np.str(tomobin)+'_plane'+np.str(plane)+'_'+np.str(gpam)+'GpAM_RA_Dec_g1_g2_w_den_Sij.asc'
        #outname = input.GalDir+'StageIV_nz/GalCat_tomo'+np.str(tomobin)+'_plane'+np.str(plane)+'_'+np.str(gpam)+'GpAM_RA_Dec_g1_g2_w_den_Sij.asc'
        outname = input.GalDir+'StageIV_nz/GalCat_tomo'+np.str(tomobin)+'_plane'+np.str(plane)+'_'+np.str(gpam)+'GpAM_RA_Dec_g1_g2_w_den_Sij_linear_bias.asc'


        print("Wrinting",outname)

        # given ra=np.degrees(phi), e2 DOES NOT NEED FLIPPING.
        #np.savetxt(outname, np.c_[ra_rand, dec_rand, shear1_pix_front*dz1 + shear1_pix_back*dz2, shear2_pix_front*dz1 + shear2_pix_back*dz2, np.ones(ngal_cur[plane]),spec_z[gal_cur],kappa_pix_front*dz1 + kappa_pix_back*dz2 ])
        np.savetxt(outname, np.c_[ra, dec, shear1, shear2, w,z, den_pix, s11_pix, s22_pix, s12_pix])



    else:
        print('found no file named '+fname)


print("Done!")

starting analysis of plane 0
found no file named IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo4_plane0_0.06GpAM_RA_Dec_g1_g2_w_linear_bias.asc
starting analysis of plane 1
found no file named IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo4_plane1_0.06GpAM_RA_Dec_g1_g2_w_linear_bias.asc
starting analysis of plane 2
found no file named IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo4_plane2_0.06GpAM_RA_Dec_g1_g2_w_linear_bias.asc
starting analysis of plane 3
found no file named IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo4_plane3_0.06GpAM_RA_Dec_g1_g2_w_linear_bias.asc
starting analysis of plane 4
found no file named IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo4_plane4_0.06GpAM_RA_Dec_g1_g2_w_linear_bias.asc
starting analysis of plane 5
found no file named IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo4_plane5_0.06GpAM_RA_Dec_g1_g2_w_linear_bias.asc
starting analysis of plane 6
found no file named IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tom

/tmp/ipykernel_56525/3280649918.py:7: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  fname = input.GalDir+'StageIV_nz/GalCat_tomo'+np.str(tomobin)+'_plane'+np.str(plane)+'_'+np.str(gpam)+'GpAM_RA_Dec_g1_g2_w_linear_bias.asc'


opening  IA-infusion/SkySim5000/density/density_map_208_dens_allsky.npy
Got density file!
opening  IA-infusion/SkySim5000/tidalfield/tidal_field_map_208_allsky_V2.npy
Got tidal file!
over density: -0.76744086 42.193733 4.7185797e-08
s11: -1.935081 4.5623956 4.780079e-05
s22: -1.9943714 4.8714604 -4.762041e-05
s12: -3.6377735 3.6846924 -1.2228373e-08


/tmp/ipykernel_56525/3280649918.py:60: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  outname = input.GalDir+'StageIV_nz/GalCat_tomo'+np.str(tomobin)+'_plane'+np.str(plane)+'_'+np.str(gpam)+'GpAM_RA_Dec_g1_g2_w_den_Sij_linear_bias.asc'


Wrinting IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo4_plane22_0.06GpAM_RA_Dec_g1_g2_w_den_Sij_linear_bias.asc
starting analysis of plane 23
Got file! Reading IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo4_plane23_0.06GpAM_RA_Dec_g1_g2_w_linear_bias.asc
opening  IA-infusion/SkySim5000/density/density_map_213_dens_allsky.npy
Got density file!
opening  IA-infusion/SkySim5000/tidalfield/tidal_field_map_213_allsky_V2.npy
Got tidal file!
over density: -0.7592609 58.152843 9.2908766e-07
s11: -1.7901424 6.392431 0.00011043918
s22: -1.8116019 6.255812 -0.00011036428
s12: -4.0580387 4.0731835 -6.7234533e-09
Wrinting IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo4_plane23_0.06GpAM_RA_Dec_g1_g2_w_den_Sij_linear_bias.asc
starting analysis of plane 24
Got file! Reading IA-infusion/SkySim5000/GalCat/StageIV_nz/GalCat_tomo4_plane24_0.06GpAM_RA_Dec_g1_g2_w_linear_bias.asc
opening  IA-infusion/SkySim5000/density/density_map_219_dens_allsky.npy
Got density file!
opening  IA-infusion

In [7]:
   random_vector = [[ 0.4854 , 0.8003],
    [0.1419  ,  0.4218],
    [0.9157  ,  0.7922],
    [0.9595  ,  0.6557]]


In [9]:
for plane in range(0,n_slices):
    print(plane, snaplist[plane], )

0 121
1 124
2 127
3 131
4 134
5 137
6 141
7 144
8 148
9 151
10 155
11 159
12 163
13 167
14 171
15 176
16 180
17 184
18 189
19 194
20 198
21 203
22 208
23 213
24 219
25 224
26 230
27 235
28 241
29 247
30 253
31 266
32 272
33 279
34 286
35 293
36 300
37 307
38 315
39 323
40 331
41 338
42 347
43 355
44 365
45 373
46 382
47 392
48 401
49 411
50 421
51 432
52 442
53 453
54 464
55 475
56 487


In [11]:
random_vector[:][0]

[0.4854, 0.8003]